In [3]:
:toolchain stable
:dep dlinoss-notebooks = { path = ".", features = ["fft"] }
//! Doc: D-LinOSS research tutorial — environment and I/O init
//! Tags: dlinoss, research, evcxr, dlinoss-notebooks, cell-1
// Purpose: Standardize kernel/toolchain and working directory for reproducible notebook runs.
// Why: Consistent CWD ensures image/data paths work regardless of how the notebook is launched.
// Safety: This cell performs file-system setup only; no model parameters are created.
// Performance: Negligible; directory creation is idempotent.
// Repro: Keep this cell as the first executable cell in the notebook.
// Feature flags: Not required here; handled later in dependency cells.
// GPU note: Device selection defaults to CPU; change later if GPU support is enabled.
// Logging: Prints toolchain, device, and CWD for quick diagnostics.
// References: See dlinoss-notebooks (re-export) for utilities.
// Doc-scan: These header comments aid later automated documentation scans.
use dlinoss_notebooks::*;
use std::fs;
let device: Device = Device::Cpu;
// Normalize working directory for file I/O (images, datasets).
set_notebook_cwd().expect("failed to set notebook cwd");
fs::create_dir_all("images_store").ok();
println!("Toolchain: stable | Device: {:?} | CWD: {}", device, std::env::current_dir().unwrap().display());

Notebook CWD set to repository root: /home/rustuser/projects/rust/active/dlinossrustcandle/notebooks
Toolchain: stable | Device: Cpu | CWD: /home/rustuser/projects/rust/active/dlinossrustcandle/notebooks


Toolchain: stable


# D-LinOSS for SSM researchers: a practical notebook

This tutorial shows how to use the D-LinOSS (Damped Linear Oscillatory State-Space) layer in Candle for research-grade signal studies and rapid experiments.

It’s written from an SSM/deep-learning research angle, not as a Rust API tour. You’ll:
- Build a D-LinOSS layer on Candle tensors (CPU).
- Generate canonical test inputs (impulse, step, sine/chirp).
- Run the layer and inspect responses (stability, shapes, quick metrics).
- See how to extend to FFT analysis (optional).

Notation (continuous-time SSM): \( \ot{x} = A x + B u,\ y = C x + D u \) with damped 2×2 oscillatory blocks for poles \(-\alpha \pm i\omega\).
Discrete-time stability comes from the exact block exponential (\(\rho(A_d)=e^{-\alpha\Delta t}<1\))).

## 1) Kernel and dependencies (evcxr)
If you run this in a Rust Jupyter kernel (evcxr), use 

- The notebook lives in `dlinossrustcandle/notebooks/`.
- Single-dependency rule: use only `:dep dlinoss-notebooks = { path = "." }`.
  - This glue crate re-exports Candle notebook utilities and the D-LinOSS API (and optional FFT helper), so no other `:dep` entries are needed.
- Optional: build the glue crate with `features = ["fft"]` to enable FFT helper usage in this notebook.


In [4]:
//! Doc: D-LinOSS research tutorial — base imports via dlinoss-notebooks glue
//! Tags: dlinoss-notebooks, imports, types, cell-3
// Purpose: Prefer a single dependency `dlinoss-notebooks` for notebooks.
// Why: It re-exports common types (Tensor, Device, DType, Result) and the D-LinOSS API.
// Safety: No side effects; imports only.
// Performance: None; compile-time only.
// Repro: Keeps notebooks consistent across the repo.
// Feature flags: Managed at crate build time; not needed here.
// GPU note: Device can be switched later if GPU is enabled in the build.
// Doc-scan: Header comments help later documentation tooling.
use dlinoss_notebooks::{Tensor, Device, DType, Result, DLinOssLayer, DLinOssLayerConfig};
// Optional helper (exists even when fft feature is off; returns empty vec then)
use dlinoss_notebooks::rfft_magnitude;

println!("✓ dlinoss-notebooks imports ready");

✓ dlinoss-notebooks imports ready


## 2) Construct a D-LinOSS layer
Choose state/input/output dims and sampling step. Default dtype is f32, device CPU.
We’ll also set a stable, reasonably excited parameterization for exploration.

In [5]:
//! Doc: Build a D-LinOSS layer with stable-ish initialization
//! Tags: dlinoss, layer, config, stability, cell-6
// Purpose: Construct a DLinOssLayer and set parameters for bounded dynamics.
// Why: Stable initialization avoids exploding responses over long horizons.
// Shapes: state_dim=m, input_dim=p, output_dim=q; dtype f32 on CPU by default.
// Safety: Pure tensor ops; no unsafe or file I/O here.
// Parameters: delta_t controls discretization step; tune for your sampling rate.
// Research: Adjust `g_const`, scaling of `b`, or layer size to explore response behavior.
// Extensibility: Make parameters trainable later during model fitting.
// Diagnostics: Prints state_dim and delta_t upon construction.
// Doc-scan: Header comments facilitate documentation extraction for research notes.
fn build_layer() -> Result<DLinOssLayer> {
    let device = Device::Cpu;
    let cfg = DLinOssLayerConfig {
        state_dim: 32,
        input_dim: 1,
        output_dim: 1,
        delta_t: 5e-3,
        dtype: DType::F32,
    };
    let mut layer = DLinOssLayer::new(cfg, &device)?;
    // Simple stable-ish init (damped band)
    let device = layer.a.device().clone();
    let m: usize = {
        let d = layer.a.dims();
        assert_eq!(d.len(), 1);
        d[0]
    };
    let g_const = 0.2f32;
    layer.g = Tensor::full(g_const, (m,), &device)?.to_dtype(DType::F32)?;
    let dt = layer.delta_t;
    let s = layer.g.affine(dt, 1.0)?;          // 1 + dt * G
    let sqrt_s = s.sqrt()?;                     // sqrt(1 + dt*G)
    let two_plus_dtg = s.affine(1.0, 1.0)?;     // 2 + dt * G
    let two_sqrt = sqrt_s.affine(2.0, 0.0)?;    // 2 * sqrt(...)
    let inv_dt2 = 1.0 / (dt * dt);
    let a_low = two_plus_dtg.sub(&two_sqrt)?.affine(inv_dt2, 0.0)?;
    let a_high = two_plus_dtg.add(&two_sqrt)?.affine(inv_dt2, 0.0)?;
    layer.a = (&a_low + &a_high)?.affine(0.5, 0.0)?; // midpoint
    layer.b = layer.b.affine(0.12, 0.0)?;            // modest drive
    Ok(layer)
}

let layer: DLinOssLayer = build_layer().expect("build_layer failed");
let m: usize = layer.a.dims()[0];
println!("layer: state_dim={}, delta_t={} s", m, layer.delta_t);

layer: state_dim=32, delta_t=0.005 s


## 3) Generate canonical inputs (impulse, step, sine, chirp)
Inputs are shaped [B, T, In]. Here we use B=1, In=1.

In [6]:
//! Doc: Signal generation utilities imported from dlinoss-notebooks
//! Tags: signals, test-inputs, impulse, step, sine, chirp, shared-utils, cell-generators
// Purpose: Use the shared SignalGen from the glue crate so notebooks don't duplicate code.
// Why: Centralizing common helpers reduces drift and ensures consistency.
// Approach: Import and rely on dlinoss-notebooks::SignalGen methods.
// Performance: Same as before; simple vector math and Tensor creation.
// Research: Impulse tests transient response; sine tests frequency response; chirp tests bandwidth.
// Extensibility: Add more signal types in the glue crate once and reuse everywhere.
// Doc-scan: Header documentation aids automated scanning tools.
use dlinoss_notebooks::SignalGen;

println!("SignalGen imported: methods available -> impulse, step, sine, chirp");

SignalGen imported: methods available -> impulse, step, sine, chirp


## 4) Run the layer and inspect outputs
We run impulse and step responses, plus a short sine. We’ll print shape summaries and quick metrics (RMS).

Fun fact: if this cell eats too many tensors, it gets gassy and blows up the font size. Don’t worry—we’ve put it on a strict diet of stable params and small T.

In [7]:
//! Doc: Forward processing with D-LinOSS layer and signal analysis
//! Tags: forward-pass, rms-metrics, signal-processing, layer-response, cell-analysis
// Purpose: Run various test signals through D-LinOSS layer and compute basic metrics.
// Why: Characterize layer behavior across different input types (transients, steady-state, periodic).
// Approach: Use a Result-returning closure to keep `?` scoped and avoid REPL never-type issues.
// Performance: Sequential processing of small test signals; efficient for characterization.
// Research: RMS values indicate energy transfer; sample previews show response dynamics.
// Extensibility: Could add frequency analysis, stability metrics, or multi-channel processing.
// Signal generation: Use shared SignalGen methods.

let result: Result<(Tensor, f32, f32, f32)> = (|| -> Result<(Tensor, f32, f32, f32)> {
    let mut layer: DLinOssLayer = build_layer()?;
    let t: usize = 64usize;
    let dt: f32 = layer.delta_t as f32;

    // Test signals using SignalGen
    let x_imp: Tensor = SignalGen::impulse(t)?;
    let x_step: Tensor = SignalGen::step(t, 0.5)?;
    let x_sine: Tensor = SignalGen::sine(t, 2.0, dt)?;

    // Forward passes
    let y_imp: Tensor = layer.forward(&x_imp, None)?;
    let y_step: Tensor = layer.forward(&x_step, None)?;
    let y_sine: Tensor = layer.forward(&x_sine, None)?;

    // Store y_sine for FFT analysis in next cells
    let y_sine_flat: Tensor = y_sine.squeeze(0)?.squeeze(1)?;

    // Compute RMS metrics directly
    let y_imp_flat: Vec<f32> = y_imp.squeeze(0)?.squeeze(1)?.to_vec1::<f32>()?;
    let rms_imp: f32 = {
        let sum_sq: f32 = y_imp_flat.iter().map(|x| x * x).sum();
        (sum_sq / y_imp_flat.len() as f32).sqrt()
    };

    let y_step_flat: Vec<f32> = y_step.squeeze(0)?.squeeze(1)?.to_vec1::<f32>()?;
    let rms_step: f32 = {
        let sum_sq: f32 = y_step_flat.iter().map(|x| x * x).sum();
        (sum_sq / y_step_flat.len() as f32).sqrt()
    };

    let y_sine_vec: Vec<f32> = y_sine_flat.to_vec1::<f32>()?;
    let rms_sine: f32 = {
        let sum_sq: f32 = y_sine_vec.iter().map(|x| x * x).sum();
        (sum_sq / y_sine_vec.len() as f32).sqrt()
    };

    println!("Forward processing results:");
    println!("  Impulse response RMS: {:.4}", rms_imp);
    println!("  Step response RMS:    {:.4}", rms_step);
    println!("  Sine response RMS:    {:.4}", rms_sine);

    // Preview first few samples
    println!("  Impulse first 5: {:?}", &y_imp_flat[0..5.min(y_imp_flat.len())]);
    println!("  Step first 5: {:?}", &y_step_flat[0..5.min(y_step_flat.len())]);
    println!("  Sine first 5: {:?}", &y_sine_vec[0..5.min(y_sine_vec.len())]);

    Ok((y_sine_flat, rms_imp, rms_step, rms_sine))
})();

let (y_sine_flat, rms_imp, rms_step, rms_sine): (Tensor, f32, f32, f32) = result.expect("forward processing failed");

Forward processing results:
  Impulse response RMS: 0.0000
  Step response RMS:    0.0000
  Sine response RMS:    0.0000
  Impulse first 5: [1.9519399e-5, 5.829172e-12, -1.9499888e-5, -1.1515965e-11, 1.9480396e-5]
  Step first 5: [9.759699e-6, 9.759702e-6, 9.758908e-9, 9.752641e-9, 9.7499515e-6]
  Sine first 5: [0.0, 1.2256332e-6, 2.44643e-6, 2.4331634e-6, 2.4102942e-6]
  Impulse response RMS: 0.0000
  Step response RMS:    0.0000
  Sine response RMS:    0.0000
  Impulse first 5: [1.9519399e-5, 5.829172e-12, -1.9499888e-5, -1.1515965e-11, 1.9480396e-5]
  Step first 5: [9.759699e-6, 9.759702e-6, 9.758908e-9, 9.752641e-9, 9.7499515e-6]
  Sine first 5: [0.0, 1.2256332e-6, 2.44643e-6, 2.4331634e-6, 2.4102942e-6]


### On stability
We use a stable parametrization (damped oscillatory blocks) so long-horizon responses remain bounded.
A simple check is that RMS doesn’t blow up as `T` grows (for bounded inputs). More formal checks can compare "before/after" energy or spectral radius in the discrete domain.

## 5) Optional: FFT-based analysis
If you enabled the `fft` feature in the :dep section, you can take short windows of the output and inspect frequency content.
Below is a minimal real-FFT magnitude sketch. If FFT is not enabled, it prints a note.

In [8]:
//! Doc: Real FFT magnitude spectrum analysis using dlinoss-notebooks
//! Tags: fft, spectral, magnitude, dlinoss-notebooks, cell-13
// Purpose: Compute spectral magnitudes from the output window using the helper function.
// Why: Frequency-domain views complement time-domain responses for oscillatory systems.
// Requirements: FFT feature enabled at crate build time; uses dlinoss-notebooks::rfft_magnitude.
// Safety: Pure numerical; tensor conversions are validated.
// Performance: FFT cost depends on window size; keep small for interactivity.
// Research: Identify resonance/damping patterns across frequency bins.
// Extensibility: Plot magnitude spectrum or compare across parameter sweeps.
// Feature-gated: This works when crate is built with --features fft (falls back to empty vec otherwise).

let fft_res: Result<()> = (|| -> Result<()> {
    let y = y_sine_flat.to_vec1::<f32>()?;
    let win = 256.min(y.len());
    let signal_window = &y[y.len()-win..];

    let magnitudes = rfft_magnitude(signal_window)?;

    if magnitudes.is_empty() {
        println!("FFT not enabled; build with --features fft for spectral analysis");
    } else {
        // Rebuild layer config just to get delta_t (sample rate)
        let tmp_layer = build_layer()?;
        let sample_rate = 1.0 / tmp_layer.delta_t as f32;  // Hz
        let freq_resolution = sample_rate / (win as f32);  // Hz per bin
        println!("FFT bins: {} | Sample rate: {:.1} Hz | Freq resolution: {:.2} Hz/bin", 
                 magnitudes.len(), sample_rate, freq_resolution);
        
        // Show first few magnitude bins
        let preview: Vec<f32> = magnitudes.iter().take(8).cloned().collect();
        println!("Magnitude spectrum (first 8 bins): {:?}", preview);
        
        // Find peak frequency
        if let Some((peak_bin, &peak_mag)) = magnitudes.iter().enumerate().max_by(|a, b| a.1.partial_cmp(b.1).unwrap()) {
            let peak_freq = peak_bin as f32 * freq_resolution;
            println!("Peak at bin {} ({:.1} Hz) with magnitude {:.4}", peak_bin, peak_freq, peak_mag);
        }
    }
    Ok(())
})();

fft_res?;

[DEBUG] cpu_fft: n = 64, batch_size = 1, stride = 1, dims = [64], input.len() = 64
FFT bins: 33 | Sample rate: 200.0 Hz | Freq resolution: 3.12 Hz/bin
Magnitude spectrum (first 8 bins): [3.1389165e-5, 2.7830505e-5, 6.7367864e-6, 3.884639e-6, 2.766933e-6, 2.1649766e-6, 1.7866902e-6, 1.5264236e-6]
Peak at bin 0 (0.0 Hz) with magnitude 0.0000
FFT bins: 33 | Sample rate: 200.0 Hz | Freq resolution: 3.12 Hz/bin
Magnitude spectrum (first 8 bins): [3.1389165e-5, 2.7830505e-5, 6.7367864e-6, 3.884639e-6, 2.766933e-6, 2.1649766e-6, 1.7866902e-6, 1.5264236e-6]
Peak at bin 0 (0.0 Hz) with magnitude 0.0000


## 6) Research extensions
- Try multiple input channels (In>1) and probe cross-coupling via `C`.
- Randomize or sweep parameterizations (e.g., vary damping and frequency bands).
- Batch processing: increase `B` to process many sequences at once.
- GPU: enable CUDA features in both Candle and the crate when needed.
- For learning: integrate with Candle optimizers and loss functions (beyond scope here), treat layer params as trainable (they’re tensors).

## 7) API quick reference for researchers

Contract and shapes:
- Input x: [B, T, In] (batch, time, input channels)
- Output y: [B, T, Out]
- Optional initial state w0: [B, 2m] where m = state_dim (concatenated position/velocity)
- DType: f32 by default (set in config)
- Device: CPU by default (set via Device::Cpu)

Construction:
- Create DLinOssLayerConfig { state_dim, input_dim, output_dim, delta_t, dtype }
- Call DLinOssLayer::new(cfg, &device)

Forward pass:
- layer.forward(&x, None) or layer.forward(&x, Some(&w0))

Stability and parameterization:
- The implementation uses a damped IMEX-inspired discretization; stability hinges on non-negative damping G and clamping A into an admissible band.
- For research sweeps, vary G (damping) and derived A within the band; monitor energy/RMS across longer T.

What you can explore:
- System ID: drive with impulse/step/sine sweeps; fit C,B (and A,G) via gradient-based learning using Candle optimizers.
- Filtering/forecasting: provide streaming inputs and retain state (use the returned final state as next w0).
- Multi-channel dynamics: set In>1/Out>1; probe cross-coupling via C and B.
- Frequency locality: with fft feature, analyze band responses; tune damping per band.
- Hardware: CPU by default; enable CUDA/Metal features in both Candle and this crate for GPU acceleration.

Notes:
- All parameters (A, G, B, C) are tensors and can be made trainable; integrate into a larger model graph with losses.
- This notebook keeps things minimal and deterministic for clarity; extend with noise, nonlinearities, or stacked layers for richer models.

## 8) Limitations and next steps
- Streaming/online use would benefit from a `forward_with_state` API returning `(y, w_T)` to carry state across segments. Today `forward` returns only `y`; for streaming you can extend the crate to expose the final state.
- Training: Wrap this layer in a larger Candle model, make `(a,g,b,c)` trainable, and connect to a task loss (e.g., forecasting MSE). Use standard optimizers (AdamW) and backprop through the scan.
- Spectral tools: Enable the crate’s `fft` feature and re-export a simple real-FFT helper (via `dlinoss_augment::TensorFftExt`) to quantify band responses.
- Multi-layer stacks: Stack D-LinOSS layers with residuals/MLPs to explore hierarchical oscillatory dynamics.

## 9) Streaming/segmented processing\nWhile the current `forward` method returns only the output sequence, you can manually simulate streaming by:\n- Running small segments sequentially.\n- Extracting the final internal state from one segment to initialize the next.\n\n**Note**: The current implementation doesn't expose the final state directly from `forward()`. For true streaming, you'd need to modify the crate to return `(output, final_state)` or add a `forward_with_state()` method. Below shows a manual approach by re-running with overlapping context."

In [ ]:
//! Doc: Manual streaming/segmented processing demonstration
//! Tags: streaming, segments, overlap, manual-state, cell-streaming
// Purpose: Show how to process long sequences in segments with overlap for pseudo-streaming.
// Why: Demonstrates segmented processing when memory or latency constraints exist.
// Approach: Use overlapping segments where the end of one provides context for the next, with a progress bar and time budget.
// Limitation: True streaming needs forward_with_state() API; this is a workaround.
// Performance: Overlap reduces efficiency but maintains causal dependencies.
// Research: Useful for online processing or when sequence length exceeds memory limits.
// Extensibility: Could be enhanced with proper state extraction from the layer.
// Doc-scan: Header comments aid documentation tooling.
// Signal generation: Use SignalGen::chirp method for long signal.
//
// What “streaming” means here:
// - We process a long input in smaller chunks (segments). Each segment overlaps a bit with the previous one
//   so dynamics have context across the boundary. This approximates online/real-time processing.
// - Because DLinOssLayer::forward currently doesn’t return the final state, we restart from zero state
//   for each segment. The overlap mitigates boundary artifacts but is not identical to true stateful streaming.
//
// Where outputs go:
// - The output samples from each segment are collected into a local Vec<f32> named `all_outputs`.
// - We skip the overlapping prefix on all but the first segment so we don’t double count samples.
// - `all_outputs` is used to compute a final RMS and then dropped at the end of the closure. It’s not persisted.
//   If you want to keep it, return it from the closure or write it to disk before returning.
//
// What to observe while it runs:
// - A single-line progress bar that updates as segments are processed and shows elapsed seconds.
// - If the time budget (~30s) is reached, the loop stops early with a message.
// - At the end, you’ll see how many output samples were produced and their RMS value. The length may be
//   slightly less than `total_length` if the final segment is shorter than the overlap (a benign truncation).
//
// About `stream_res?;` at the bottom:
// - The closure returns a Result<()> that we bind to `stream_res`. The trailing `?` tells evcxr to surface any
//   error nicely at the notebook level (it’s equivalent to `stream_res.unwrap()` but preserves error messages).
// - On success, it’s effectively a no-op. It doesn’t “stream” to a destination; printing inside the closure is
//   what you see in the cell output.

let stream_res: Result<()> = (|| -> Result<()> {
    use std::io::{self, Write};
    use std::time::{Duration, Instant};
    
    let mut layer: DLinOssLayer = build_layer()?;
    let total_length: usize = 4096usize; // keep reasonably sized for tutorial speed
    let segment_size: usize = 512usize;  // larger segments -> fewer iterations
    let overlap: usize = 64usize;        // small overlap for context
    
    // Time budget and progress bar settings
    let start_time: Instant = Instant::now();
    let max_duration: Duration = Duration::from_secs(30); // cap ~30s
    let bar_width: usize = 40;
    
    // Generate a long chirp signal for segmented processing using SignalGen
    let dt: f32 = layer.delta_t as f32;
    let x_long: Tensor = SignalGen::chirp(total_length, 0.5, 10.0, dt)?;
    let x_long_flat: Vec<f32> = x_long.squeeze(0)?.squeeze(1)?.to_vec1::<f32>()?;
    
    let mut all_outputs: Vec<f32> = Vec::with_capacity(total_length);
    let mut start_idx: usize = 0;
    
    while start_idx < total_length {
        let end_idx: usize = (start_idx + segment_size).min(total_length);
        let segment_len: usize = end_idx - start_idx;
        
        // Extract segment from the long input (shape it back to [B=1, T=segment_len, In=1])
        let segment_data: &[f32] = &x_long_flat[start_idx..end_idx];
        let x_segment: Tensor = Tensor::from_slice(segment_data, (1, segment_len, 1), &Device::Cpu)?;
        
        // Process segment (no initial state - starts fresh each time)
        // In a true streaming implementation, we’d pass the previous final state
        let y_segment: Tensor = layer.forward(&x_segment, None)?;
        let y_segment_flat: Vec<f32> = y_segment.squeeze(0)?.squeeze(1)?.to_vec1::<f32>()?;
        
        // Store output (skip overlap region except for first segment)
        let store_start: usize = if start_idx == 0 { 0 } else { overlap };
        all_outputs.extend_from_slice(&y_segment_flat[store_start..]);
        
        // Progress reporting: prints a single carriage-returned line that updates in-place
        let processed: usize = end_idx;
        let pct: f32 = (processed as f32 / total_length as f32) * 100.0;
        let filled_len: usize = ((pct / 100.0) * bar_width as f32) as usize;
        let mut bar: String = String::with_capacity(bar_width);
        for _ in 0..filled_len { bar.push('#'); }
        for _ in filled_len..bar_width { bar.push('-'); }
        let elapsed: f32 = start_time.elapsed().as_secs_f32();
        print!("\rProgress: [{bar}] {pct:5.1}% | elapsed {elapsed:5.1}s");
        io::stdout().flush().ok();
        
        // Move to next segment with overlap (ensures continuity across boundaries)
        start_idx = end_idx.saturating_sub(overlap);
        
        // Respect time budget
        if start_time.elapsed() >= max_duration {
            println!("\nTime budget reached (~30s). Stopping early at ~{pct:.1}%.");
            break;
        }
    }
    println!(); // newline after progress bar
    
    // Summarize output energy for a quick health check
    let final_rms: f32 = {
        let sum_sq: f32 = all_outputs.iter().map(|x| x * x).sum();
        (sum_sq / all_outputs.len() as f32).sqrt()
    };
    
    println!("Segmented processing complete: {} total output samples, RMS = {:.4}", 
             all_outputs.len(), final_rms);
    println!("Note: For true streaming, extend the crate with forward_with_state() API");
    Ok(())
})();

// Surface any error from the closure at the notebook level; no-op on success
stream_res?;

Progress: [########################################] 100.0% | elapsed  30.0s
Time budget reached (~30s). Stopping early at ~100.0%.

Segmented processing complete: 4096 total output samples, RMS = 0.0000
Note: For true streaming, extend the crate with forward_with_state() API
